In [1]:
import sys
sys.path.append("..")

import copy
import cProfile
from datasets import load_dataset
import math
import nltk
import pandas as pd
import random
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, DataCollatorWithPadding, \
                         AutoModelForSequenceClassification, BertForSequenceClassification

from resilient_nlp.mini_roben import Clustering, ClusterRepRecoverer, ClusterRecovererWithPassthrough
from resilient_nlp.models import BertClassifier
from resilient_nlp.perturbers import ToyPerturber, WordScramblerPerturber
from runner import ExperimentRunner
from word_score_attack import BertWordScoreAttack

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
# imdb = load_dataset('imdb')
imdb = load_dataset('artemis13fowl/imdb')

Using custom data configuration artemis13fowl--imdb-f63738dec0d5e230
Reusing dataset parquet (/home/scpdxcs/.cache/huggingface/datasets/parquet/artemis13fowl--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
random.seed(11)
sampled_test_set = imdb['attack_eval_truncated']

# This is silly but apparently huggingface datasets are immutable?
# Representing it as something a bit more sane
sampled_test_set_dict = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set
]

In [4]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
checkpoint_finetuned = "artemis13fowl/bert-base-uncased-imdb"
model_finetuned = BertForSequenceClassification.from_pretrained(checkpoint_finetuned).to(device)

In [6]:
roben_clustering = Clustering.from_pickle("../vocab100000_ed1.pkl")
roben_recoverer = ClusterRecovererWithPassthrough("cache", roben_clustering)
roben_clustering2 = Clustering.from_pickle("../vocab100000_ed1_gamma0.3.pkl")
roben_recoverer2 = ClusterRecovererWithPassthrough("cache", roben_clustering2)

In [7]:
random.seed(11)
sampled_test_set_adv_no_ws = []
wsp = WordScramblerPerturber(perturb_prob=0.1, weight_add=1, weight_drop=1, weight_swap=1,
                             weight_split_word=0, weight_merge_words=0)

for i in range(10):
    test_item = copy.deepcopy(sampled_test_set_dict)

    for row in test_item:
        row['text'] = wsp.perturb([row['text']])[0][0]
    sampled_test_set_adv_no_ws.append(test_item)

In [8]:
random.seed(11)
sampled_test_set_adv_incl_ws = []
wsp = WordScramblerPerturber(perturb_prob=0.1, weight_add=1, weight_drop=1, weight_swap=1,
                             weight_split_word=1, weight_merge_words=1)

for i in range(10):
    test_item = copy.deepcopy(sampled_test_set_dict)

    for row in test_item:
        row['text'] = wsp.perturb([row['text']])[0][0]
    sampled_test_set_adv_incl_ws.append(test_item)

In [9]:
max_sequence_length = 128
batch_size = 32
eval_steps = 100

In [10]:
def standard_model_predict(tokenizer, model, sentences, recoverer, return_pred_tensor, recoverer_tokenize):
    if recoverer is not None:
        if recoverer_tokenize:
            tok = nltk.tokenize.treebank.TreebankWordTokenizer()
            sentences = [ " ".join(tok_list) for tok_list in tok.tokenize_sents(sentences) ]
        sentences = [ recoverer.recover(s.lower()) for s in sentences ]
        if recoverer_tokenize:
            detok = nltk.tokenize.treebank.TreebankWordDetokenizer()
            sentences = [ detok.detokenize(s.split(" ")) for s in sentences]
    tokenized = tokenizer(sentences, truncation=True, padding='max_length', max_length=max_sequence_length,
                          return_tensors='pt')
    tokenized = { k: v.to(device) for k, v in tokenized.items() }
    preds = model(**tokenized)
    if return_pred_tensor:
        return preds
    else:
        return torch.argmax(preds.logits, dim=1)

def wrap_standard_model(tokenizer, model, recoverer=None, return_pred_tensor=True, recoverer_tokenize=False):
    return lambda sentences: standard_model_predict(tokenizer, model, sentences, recoverer, return_pred_tensor,
                                                    recoverer_tokenize)

In [11]:
def mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding, pad_embedding, sentences, return_pred_tensor):
    # Truncate and lower case. Truncation is for performance only
    sentences = [ s.lower()[:5*max_sequence_length] for s in sentences]
    embedding = runner.embed(sentences=sentences,
        start_token=cls_embedding, end_token=sep_embedding, pad_token=pad_embedding,
        max_tokens=max_sequence_length)
    preds = model(inputs_embeds=embedding['inputs_embeds'], attention_mask=embedding['attention_mask'])
    if return_pred_tensor:
        return preds
    else:
        return torch.argmax(preds.logits, dim=1)

def wrap_mltokenizer_model(mltokenizer_prefix, tokenizer, model, return_pred_tensor=True):
    filename = "../{}.pth".format(mltokenizer_prefix)
    runner = ExperimentRunner(device, model_filename=filename)
    cf_embedding = model.base_model.embeddings.word_embeddings
    cls_token_id = tokenizer.vocab['[CLS]']
    sep_token_id = tokenizer.vocab['[SEP]']
    pad_token_id = tokenizer.vocab['[PAD]']
    cls_embedding = cf_embedding(torch.tensor([cls_token_id], device=device)).view(-1)
    sep_embedding = cf_embedding(torch.tensor([sep_token_id], device=device)).view(-1)
    pad_embedding = cf_embedding(torch.tensor([pad_token_id], device=device)).view(-1)
    
    return lambda sentences: mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding,
                                                      pad_embedding, sentences, return_pred_tensor)

In [12]:
@torch.no_grad()
def evaluate_model(model, test_set):
    num_batches = math.ceil(len(test_set) / batch_size)
    
    sentences = [ x['text'] for x in test_set ]
    labels = [ x['label'] for x in test_set ]
    pred_batches = []
    
    for i in tqdm(range(num_batches)):
        bs = i * batch_size
        be = bs + batch_size
        
        output = model(sentences[bs:be])
        
        pred_batches.append(torch.argmax(output.logits, dim=1).detach().cpu())
    preds = torch.cat(pred_batches)
    
    print(classification_report(labels, preds, digits=4))
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    
    return accuracy, f1

In [13]:
@torch.no_grad()
def evaluate_model_adv(model, test_sets):
    labels = [ x['label'] for x in test_sets[0] ]
    adv_preds = copy.copy(labels)
    accuracy_list = []
    f1_list = []
    
    for idx in tqdm(range(len(test_sets))):
        test_set = test_sets[idx]
        num_batches = math.ceil(len(test_set) / batch_size)
    
        sentences = [ x['text'] for x in test_set ]
        pred_batches = []
    
        for i in range(num_batches):
            bs = i * batch_size
            be = bs + batch_size
        
            output = model(sentences[bs:be])
        
            pred_batches.append(torch.argmax(output.logits, dim=1).detach().cpu())
        preds = torch.cat(pred_batches)
        
        for i in range(len(adv_preds)):
            if labels[i] == 1.0 and preds[i] == 0.0:
                adv_preds[i] = 0.0
            elif labels[i] == 0.0 and preds[i] == 1.0:
                adv_preds[i] = 1.0

        accuracy_list.append(accuracy_score(labels, adv_preds))
        f1_list.append(f1_score(labels, adv_preds, average='macro'))
    
    print(classification_report(labels, adv_preds, digits=4))    
    
    return accuracy_list, f1_list

In [14]:
@torch.no_grad()
def evaluate_model_word_score(model, test_set, allow_whitespace_pert=True):
    attacker = BertWordScoreAttack(
        WordScramblerPerturber(perturb_prob=1, weight_add=1, weight_drop=1, weight_swap=1,
                               weight_split_word=int(allow_whitespace_pert),
                               weight_merge_words=int(allow_whitespace_pert)),
        "../output/imdb_word_scores.json", model, tokenizer=None, max_sequence_length=max_sequence_length
    )

    res = attacker.attack(test_set, max_tokens_to_query=10, max_tries_per_token=2, mode=0, print_summary=False)
    
    print(classification_report(res['ground_truth'], res['perturbed_preds'], digits=4))    
    
    accuracy = accuracy_score(res['ground_truth'], res['perturbed_preds'])
    f1 = f1_score(res['ground_truth'], res['perturbed_preds'], average='macro')
    
    return accuracy, f1

In [15]:
baseline_model = wrap_standard_model(tokenizer, model_finetuned)

In [16]:
mltok_model = wrap_mltokenizer_model('output/64k_lstm_all_pert_finetuned', tokenizer, model_finetuned)

In [17]:
baseline_roben_model = wrap_standard_model(tokenizer, model_finetuned, roben_recoverer)

In [18]:
baseline_roben_model_tok = wrap_standard_model(tokenizer, model_finetuned, roben_recoverer, recoverer_tokenize=True)

In [19]:
baseline_roben_model2 = wrap_standard_model(tokenizer, model_finetuned, roben_recoverer2)

In [20]:
baseline_roben_model2_tok = wrap_standard_model(tokenizer, model_finetuned, roben_recoverer2, recoverer_tokenize=True)

In [21]:
all_models = {
    'baseline': baseline_model,
    'roben_1': baseline_roben_model,
    'roben_2': baseline_roben_model2,
    'roben_1_tok': baseline_roben_model,
    'roben_2_tok': baseline_roben_model2_tok,
}

mltok_model_names = [
    '64k_lstm_clean_vanilla',
    '64k_lstm_no_whitespace_pert_vanilla',
    '64k_lstm_all_pert_vanilla',
    '64k_lstm_clean_finetuned',
    '64k_lstm_no_whitespace_pert_finetuned',
    '64k_lstm_all_pert_finetuned',
    '64k_cnn_no_whitespace_pert_finetuned',
]

for name in mltok_model_names:
    all_models[name] = wrap_mltokenizer_model(f'output/{name}', tokenizer, model_finetuned)

In [22]:
evaluations = [
    'clean',
    'stochastic_no_ws',
    'stochastic_incl_ws',
    'word_score_no_ws',
    'word_score_incl_ws',
]

In [23]:
accuracy_df = pd.DataFrame(columns=evaluations, index=all_models.keys())
f1_df = pd.DataFrame(columns=evaluations, index=all_models.keys())

for cur_model_name, cur_model in all_models.items():
    for cur_evaluation in evaluations:
        print(f'Evaluating model {cur_model_name} on {cur_evaluation}')
        random.seed(11)
        if cur_evaluation == 'clean':
            acc, f1 = evaluate_model(cur_model, sampled_test_set)
        elif cur_evaluation.startswith('stochastic_'):
            if cur_evaluation == 'stochastic_no_ws':
                acc_list, f1_list = evaluate_model_adv(cur_model, sampled_test_set_adv_no_ws)
            elif cur_evaluation == 'stochastic_incl_ws':
                acc_list, f1_list = evaluate_model_adv(cur_model, sampled_test_set_adv_incl_ws)
            acc = acc_list[-1]
            f1 = f1_list[-1]
        elif cur_evaluation.startswith('word_score_'):
            if cur_evaluation == 'word_score_no_ws':
                acc, f1 = evaluate_model_word_score(cur_model, sampled_test_set, allow_whitespace_pert=False)
            elif cur_evaluation == 'word_score_incl_ws':
                acc, f1 = evaluate_model_word_score(cur_model, sampled_test_set, allow_whitespace_pert=True)

        accuracy_df[cur_evaluation][cur_model_name] = acc
        f1_df[cur_evaluation][cur_model_name] = f1

Evaluating model baseline on clean


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.04it/s]


              precision    recall  f1-score   support

           0     0.8765    0.8765    0.8765       243
           1     0.8833    0.8833    0.8833       257

    accuracy                         0.8800       500
   macro avg     0.8799    0.8799    0.8799       500
weighted avg     0.8800    0.8800    0.8800       500

Evaluating model baseline on stochastic_no_ws


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:16<00:00,  7.67s/it]


              precision    recall  f1-score   support

           0     0.6970    0.7572    0.7258       243
           1     0.7500    0.6887    0.7181       257

    accuracy                         0.7220       500
   macro avg     0.7235    0.7230    0.7219       500
weighted avg     0.7242    0.7220    0.7218       500

Evaluating model baseline on stochastic_incl_ws


...

              precision    recall  f1-score   support

           0     0.1333    0.0082    0.0155       243
           1     0.5031    0.9494    0.6577       257

    accuracy                         0.4920       500
   macro avg     0.3182    0.4788    0.3366       500
weighted avg     0.3234    0.4920    0.3456       500

Evaluating model 64k_cnn_no_whitespace_pert_finetuned on word_score_no_ws


500it [08:26,  1.01s/it]


              precision    recall  f1-score   support

           0     0.3333    0.0041    0.0081       243
           1     0.5131    0.9922    0.6764       257

    accuracy                         0.5120       500
   macro avg     0.4232    0.4982    0.3423       500
weighted avg     0.4257    0.5120    0.3516       500

Evaluating model 64k_cnn_no_whitespace_pert_finetuned on word_score_incl_ws


500it [08:24,  1.01s/it]

              precision    recall  f1-score   support

           0     0.2000    0.0082    0.0158       243
           1     0.5082    0.9689    0.6667       257

    accuracy                         0.5020       500
   macro avg     0.3541    0.4886    0.3412       500
weighted avg     0.3584    0.5020    0.3504       500



In [24]:
accuracy_df

,clean,stochastic_no_ws,stochastic_incl_ws,word_score_no_ws,word_score_incl_ws
baseline,0.88,0.722,0.69,0.644,0.652
roben_1,0.794,0.686,0.572,0.68,0.618
roben_2,0.818,0.71,0.584,0.724,0.668
roben_1_tok,0.794,0.686,0.572,0.68,0.618
roben_2_tok,0.786,0.684,0.59,0.734,0.65
64k_lstm_clean_vanilla,0.864,0.708,0.662,0.68,0.624
64k_lstm_no_whitespace_pert_vanilla,0.872,0.812,0.712,0.776,0.758
64k_lstm_all_pert_vanilla,0.868,0.81,0.802,0.782,0.77
64k_lstm_clean_finetuned,0.864,0.724,0.694,0.694,0.66
64k_lstm_no_whitespace_pert_finetuned,0.864,0.816,0.722,0.788,0.732


In [25]:
f1_df

,clean,stochastic_no_ws,stochastic_incl_ws,word_score_no_ws,word_score_incl_ws
baseline,0.879906,0.721946,0.689989,0.643538,0.651978
roben_1,0.792002,0.683201,0.568959,0.67397,0.60968
roben_2,0.817104,0.708881,0.582289,0.721212,0.665862
roben_1_tok,0.792002,0.683201,0.568959,0.67397,0.60968
roben_2_tok,0.784092,0.680808,0.587224,0.730244,0.644354
64k_lstm_clean_vanilla,0.863922,0.707981,0.661771,0.67998,0.624
64k_lstm_no_whitespace_pert_vanilla,0.871926,0.811892,0.711959,0.775565,0.757922
64k_lstm_all_pert_vanilla,0.867788,0.809597,0.801504,0.781454,0.769424
64k_lstm_clean_finetuned,0.863946,0.72396,0.693969,0.693852,0.659951
64k_lstm_no_whitespace_pert_finetuned,0.863965,0.815988,0.721999,0.787783,0.731725


In [26]:
accuracy_df.to_csv("../output/grid_accuracy.csv")

In [27]:
f1_df.to_csv("../output/grid_f1.csv")